In [11]:
using Cbc
using JuMP


In [12]:
D = [2 0;1 3] #produit par demande
C = [1 1;2 3;3 2] #prix produit par magazin
B = [2.5 1;1 2;2 1] #disponibilité produit par magazin en stock
N = 2 #nombre de genres de produits
M = 3 #nombre de magazins
F = 2 #nombre de demandes
Z = [1 0 0;0 2 1] # cout de livraison du colis 
A = 10000 #on choisit une constante qui est certainement >=Max(x+y)


10000

In [13]:
# set optimizer
m = Model(Cbc.Optimizer)

# define variables
@variable(m, x[1:N, 1:M, 1:F] >= 0,Int)
@variable(m, y[1:F,1:M],Bin)

#difinir la fonction objective
@objective(m, Min, sum(y[k,j]Z[k,j] for k = 1:F,j = 1:M) + sum(sum(x[i,j,k])*C[j,i] for i in 1:N, j=1:M, k =1:F))



y[1,1] + 2 y[2,2] + y[2,3] + x[1,1,1] + x[1,1,2] + 2 x[1,2,1] + 2 x[1,2,2] + 3 x[1,3,1] + 3 x[1,3,2] + x[2,1,1] + x[2,1,2] + 3 x[2,2,1] + 3 x[2,2,2] + 2 x[2,3,1] + 2 x[2,3,2]

In [18]:
#definir les contraintes
#disponibilité des produits en stocks
for i = 1:N,j = 1:M
       @constraint(m,sum(x[i,j,:])<=B[j,i]) 
end

#Contrainte concernant les demandes
for i in 1:N, k = 1:F
    @constraint(m,sum(x[i,j,k] for j in 1:M) == D[k,i])
end

#Contrainte de la méthode Big_M, rappel A est notre constante M dans ce code
for k = 1:F,j = 1:M
    @constraint(m,sum(x[i,j,k] for i in 1:N) - A*y[k,j] <= 0)#contrainte big M
end




In [19]:
# run optimization
optimize!(m)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 9.50045 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 1 strengthened rows, 0 substitutions
Cgl0004I processed model has 8 rows, 11 columns (11 integer (8 of which binary)) and 22 elements
Cbc0012I Integer solution of 14 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0001I Search completed - best objective 14, took 0 iterations and 0 nodes (0.02 seconds)
Cbc0035I Maximum depth 0, 3 variables fixed on reduced cost
Cuts at root node changed objective from 13 to 14
Probing was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Gomory was tried 0 times and created 0 cuts of which 0 were active after adding rounds of cuts (0.000 seconds)
Knapsack was tried 0 times and c

In [20]:
#Affichage

println("dépense minimale pour le gestionnaire des magazins = $(objective_value(m)) ")

for i in 1:N, j=1:M, k=1:F
    println("\t quantite de fluide $i par magazin $j par client $k = ",getvalue(x[i,j,k]))
      
end

dépense minimale pour le gestionnaire des magazins = 14.0 
	 quantite de fluide 1 par magazin 1 par client 1 = 2.0
	 quantite de fluide 1 par magazin 1 par client 2 = 0.0
	 quantite de fluide 1 par magazin 2 par client 1 = 0.0
	 quantite de fluide 1 par magazin 2 par client 2 = 1.0
	 quantite de fluide 1 par magazin 3 par client 1 = 0.0
	 quantite de fluide 1 par magazin 3 par client 2 = 0.0
	 quantite de fluide 2 par magazin 1 par client 1 = 0.0
	 quantite de fluide 2 par magazin 1 par client 2 = 1.0
	 quantite de fluide 2 par magazin 2 par client 1 = 0.0
	 quantite de fluide 2 par magazin 2 par client 2 = 1.0
	 quantite de fluide 2 par magazin 3 par client 1 = 0.0
	 quantite de fluide 2 par magazin 3 par client 2 = 1.0
